# 房价预测 - PyCaret回归实现

> 🎯 使用PyCaret快速建立房价预测模型（回归任务）

---

In [ ]:
# 环境准备
from pycaret.regression import *
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import fetch_california_housing

# 加载加州房价数据集
housing = fetch_california_housing(as_frame=True)
data = housing.frame

print(f"数据形状: {data.shape}")
print(f"\n特征列: {housing.feature_names}")
print(f"\n目标列: {housing.target_names}")
data.head()

In [ ]:
# 数据概览
data.describe()

## PyCaret初始化

In [ ]:
# 初始化回归环境
reg = setup(
    data=data,
    target='MedHouseVal',
    session_id=42,
    
    # 预处理
    normalize=True,              # 标准化
    transformation=True,         # 特征变换
    transform_target=True,       # 目标变量变换（重要！）
    
    # 特征工程
    feature_interaction=True,    # 特征交互
    polynomial_features=True,    # 多项式特征
    
    # 其他
    remove_outliers=True,        # 移除异常值（回归任务重要）
    outliers_threshold=0.05,     # 异常值阈值
    verbose=True
)

## 模型对比

In [ ]:
# 对比所有回归模型
best_model = compare_models(
    sort='R2',              # 按R²排序
    n_select=1
)

In [ ]:
# 调优
tuned_model = tune_model(
    best_model,
    optimize='R2',
    n_iter=30
)

## 可视化分析

In [ ]:
# 残差图
plot_model(tuned_model, plot='residuals')

In [ ]:
# 预测误差图
plot_model(tuned_model, plot='error')

In [ ]:
# 特征重要性
plot_model(tuned_model, plot='feature')

In [ ]:
# 学习曲线
plot_model(tuned_model, plot='learning')

## 模型融合

In [ ]:
# 获取前3个模型
top3 = compare_models(n_select=3, sort='R2')

# Stacking集成
stacked = stack_models(top3, optimize='R2')

## 测试集评估

In [ ]:
# 预测
predictions = predict_model(tuned_model)

# 评估指标
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error

y_test = predictions['MedHouseVal']
y_pred = predictions['prediction_label']

r2 = r2_score(y_test, y_pred)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
mae = mean_absolute_error(y_test, y_pred)

print(f"R² Score: {r2:.4f}")
print(f"RMSE: {rmse:.4f}")
print(f"MAE: {mae:.4f}")

In [ ]:
# 可视化预测vs实际
plt.figure(figsize=(10, 6))
plt.scatter(y_test, y_pred, alpha=0.5)
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--', lw=2)
plt.xlabel('Actual Price')
plt.ylabel('Predicted Price')
plt.title(f'Prediction vs Actual (R²={r2:.4f})')
plt.tight_layout()
plt.show()

## 保存模型

In [ ]:
# 最终化模型
final_model = finalize_model(tuned_model)

# 保存
save_model(final_model, 'pycaret_house_price_model')

print("✅ 模型已保存")

## 总结

### PyCaret回归特点

1. **目标变换**: `transform_target=True` 很重要
2. **异常值处理**: 回归任务对异常值敏感
3. **多项式特征**: 提升非线性拟合能力
4. **模型融合**: Stacking通常能提升1-3%

### 关键发现

- **最重要特征**: MedInc（收入中位数）
- **最佳模型**: 通常是XGBoost或LightGBM
- **性能**: R²通常在0.63-0.65之间

**下一步**: 查看 `02_flaml_regression.ipynb`